In [1]:
import amrlib
stog = amrlib.load_stog_model('/home/david/tmp/model_parse_xfm_bart_base-v0_1_0')
graphs = stog.parse_sents(["Japan's Defence Minister Nobuo Kishi said first missile flew about 300km (186 miles) with a maximum altitude of around 550 km, while the second, reaching as high as 50 km, travelled around 750 km.e."])
for graph in graphs:
    print(graph)
    print('\n\n')


# ::snt Japan's Defence Minister Nobuo Kishi said first missile flew about 300km (186 miles) with a maximum altitude of around 550 km, while the second, reaching as high as 50 km, travelled around 750 km.e.
(s / say-01
      :ARG0 (p / person
            :name (n / name
                  :op1 "Nobuo"
                  :op2 "Kishi")
            :ARG0-of (h / have-org-role-91
                  :ARG1 (c / country
                        :name (n2 / name
                              :op1 "Japan"))
                  :ARG2 (m / minister
                        :topic (d / defend-01))))
      :ARG1 (c2 / contrast-01
            :ARG1 (f / fly-01
                  :ARG1 (m2 / missile
                        :ord (o / ordinal-entity
                              :value 1)
                        :ARG0-of (h2 / have-03
                              :ARG1 (a / altitude
                                    :mod (m3 / maximum)
                                    :quant (a2 / around
                

In [17]:
import penman

a = penman.decode(graphs[0])

print(a.metadata,'\n\n')

print(a.triples, '\n\n')

print(a.edges(), '\n\n')

print(a.attributes(), '\n\n')




{'snt': "Japan's Defence Minister Nobuo Kishi said first missile flew about 300km (186 miles) with a maximum altitude of around 550 km, while the second, reaching as high as 50 km, travelled around 750 km.e."} 


[('s', ':instance', 'say-01'), ('s', ':ARG0', 'p'), ('p', ':instance', 'person'), ('p', ':name', 'n'), ('n', ':instance', 'name'), ('n', ':op1', '"Nobuo"'), ('n', ':op2', '"Kishi"'), ('h', ':ARG0', 'p'), ('h', ':instance', 'have-org-role-91'), ('h', ':ARG1', 'c'), ('c', ':instance', 'country'), ('c', ':name', 'n2'), ('n2', ':instance', 'name'), ('n2', ':op1', '"Japan"'), ('h', ':ARG2', 'm'), ('m', ':instance', 'minister'), ('m', ':topic', 'd'), ('d', ':instance', 'defend-01'), ('s', ':ARG1', 'c2'), ('c2', ':instance', 'contrast-01'), ('c2', ':ARG1', 'f'), ('f', ':instance', 'fly-01'), ('f', ':ARG1', 'm2'), ('m2', ':instance', 'missile'), ('m2', ':ord', 'o'), ('o', ':instance', 'ordinal-entity'), ('o', ':value', '1'), ('h2', ':ARG0', 'm2'), ('h2', ':instance', 'have-03'), ('h2'

In [10]:
import dgl
import torch

In [20]:
nodes = set()
edge_types = set()


for edge in a.edges():
    source, role, target = edge.source, edge.role, edge.target
    nodes.add(source)
    nodes.add(target)
    # v stands for vertex - not sure if we need multiple vertices in the future.
    edge_types.add(('v',role,'v'))

for attribute in a.attributes():
    source, role, target = attribute.source, attribute.role, attribute.target
    nodes.add(source)
    nodes.add(target)
    edge_types.add(('v',role,'v'))

print(nodes)
print(edge_types)



{'"Japan"', 'f', 'p', 'a2', 'o2', 'n', 's', '"Kishi"', 'k3', 'm3', 'n2', 't', 'h4', 'a', 'e', 'o', 'h2', '550', '"Nobuo"', '50', '1', 'c', 'r', 'd4', 'k2', 'c2', 'm2', 'h', '750', 'k', 'a3', '2', 'd2', 'd', 'd3', 'm4', 'h3', 'm'}
{('v', ':ARG0', 'v'), ('v', ':ARG3', 'v'), ('v', ':name', 'v'), ('v', ':ARG4', 'v'), ('v', ':op2', 'v'), ('v', ':unit', 'v'), ('v', ':value', 'v'), ('v', ':quant', 'v'), ('v', ':op1', 'v'), ('v', ':topic', 'v'), ('v', ':mod', 'v'), ('v', ':ARG1', 'v'), ('v', ':ord', 'v'), ('v', ':ARG2', 'v')}


In [29]:
edge_collection = {}

for edge_type in edge_types:
    # v1, t, v2  = edge_type
    edge_collection[edge_type] = []

print(edge_collection)

node_to_index = dict(list(map(lambda j : (j[1],j[0]), enumerate(list(nodes)))))

print(node_to_index)

graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}

g = dgl.heterograph(graph_data)

print(g)


{('v', ':ARG0', 'v'): [], ('v', ':ARG3', 'v'): [], ('v', ':name', 'v'): [], ('v', ':ARG4', 'v'): [], ('v', ':op2', 'v'): [], ('v', ':unit', 'v'): [], ('v', ':value', 'v'): [], ('v', ':quant', 'v'): [], ('v', ':op1', 'v'): [], ('v', ':topic', 'v'): [], ('v', ':mod', 'v'): [], ('v', ':ARG1', 'v'): [], ('v', ':ord', 'v'): [], ('v', ':ARG2', 'v'): []}
{'"Japan"': 0, 'f': 1, 'p': 2, 'a2': 3, 'o2': 4, 'n': 5, 's': 6, '"Kishi"': 7, 'k3': 8, 'm3': 9, 'n2': 10, 't': 11, 'h4': 12, 'a': 13, 'e': 14, 'o': 15, 'h2': 16, '550': 17, '"Nobuo"': 18, '50': 19, '1': 20, 'c': 21, 'r': 22, 'd4': 23, 'k2': 24, 'c2': 25, 'm2': 26, 'h': 27, '750': 28, 'k': 29, 'a3': 30, '2': 31, 'd2': 32, 'd': 33, 'd3': 34, 'm4': 35, 'h3': 36, 'm': 37}
Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'trea

In [ ]:
import torch
!pip install dgl

import dgl



data_dict = {
    ('user', 'edge', 'user'): (torch.tensor([0, 1]), torch.tensor([1, 2])),
    ('user', 'attribute', 'topic'): (torch.tensor([1, 1]), torch.tensor([1, 2])),
    ('user', 'plays', 'game'): (torch.tensor([0, 3]), torch.tensor([3, 4]))

}

g = dgl.heterograph(data_dict)

g

In [ ]:

i = 100

graphs = stog.parse_sents([dataset['train'][i]['premise'], dataset['train'][i]['hypothesis']])

for graph in graphs:
    print(graph)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hans")

In [9]:
from datasets import load_dataset

dataset = load_dataset("hans")

Reusing dataset hans (/home/david/.cache/huggingface/datasets/hans/plain_text/1.0.0/452e93cf5383f5ae39088254215b517d0da98ccaaf0af8f7ab04d8f23f67dbd9)


  0%|          | 0/2 [00:00<?, ?it/s]